# Setup

## Install all packages and its dependencies

In [70]:
!pip install pg8000==1.31.1 \
SQLAlchemy==2.0.29 \
google-cloud-aiplatform==1.46.0 \
google-cloud-alloydb-connector==1.0.0 --upgrade --quiet

## Enable required resources

In [71]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable alloydb.googleapis.com
!gcloud services enable compute.googleapis.com
!gcloud services enable servicenetworking.googleapis.com
!gcloud services enable cloudresourcemanager.googleapis.com

## Define google cloud project

In [72]:
import sys

PROJECT_ID = "rag-project-425902"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# if not running on colab, try to get the PROJECT_ID automatically
if "google.colab" not in sys.modules:
    import subprocess

    PROJECT_ID = subprocess.check_output(
        ["gcloud", "config", "get-value", "project"], text=True
    ).strip()

print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: rag-project-425902


In [73]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Import libraries

In [5]:
import subprocess
from typing import Tuple

import pandas as pd
import pg8000
import sqlalchemy
import vertexai
from google.cloud.alloydb.connector import Connector
from sqlalchemy.engine import Engine
from sqlalchemy.exc import DatabaseError
from vertexai.generative_models import (GenerationConfig, 
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part)
from langchain_core.prompts import PromptTemplate

In [6]:
EMBEDDING_MODEL = "textembedding-gecko@003"  # @param {type:"string"}
DIMENSIONS = 768  # @param {type:"integer"}
GENERATIVE_MODEL = "gemini-1.0-pro"  # @param {type:"string"}

## Load gemini & embeddings
- 2 models will be generated here: 1 for RAG and one for non-RAG purposes

#### RAG model

In [7]:
from langchain.llms import VertexAI


# Text model instance integrated with langChain
llm = VertexAI(
    # model_name="text-bison@002",
    model_name = GENERATIVE_MODEL,
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
)


# Embeddings API integrated with langChain
# embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003")
# DIMENSIONS = 768  # @param {type:"integer"}

/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VertexAI` was deprecated in LangChain 0.0.12 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


#### non-RAG model

In [8]:
# Set model parameters
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

non_rag_model = GenerativeModel(GENERATIVE_MODEL)

In [9]:
# Set safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}

## Link to cluster and instance on alloyDB

In [10]:
REGION = "us-central1"  # @param {type:"string"}
CPU_COUNT = 2  # @param {type:"integer"}

PROJECT_NUM = (
    subprocess.check_output(
        [
            "gcloud",
            "projects",
            "describe",
            PROJECT_ID,
            "--format",
            "value(projectNumber)",
        ]
    )
    .decode("utf-8")
    .strip()
)
print(f"Project Number: {PROJECT_NUM}")

SERVICE_ACCOUNT = (
    f"serviceAccount:service-{PROJECT_NUM}@gcp-sa-alloydb.iam.gserviceaccount.com"
)
print(f"AlloyDB Service Agent: {SERVICE_ACCOUNT}")

Project Number: 3982497274
AlloyDB Service Agent: serviceAccount:service-3982497274@gcp-sa-alloydb.iam.gserviceaccount.com


## Load marketing content

In [94]:
# df = pd.read_csv('test.csv')
df = pd.read_excel('marketing_template.xlsx', sheet_name='Sheet1')
df.head(5)

template_id                description  \
0        12345       Investment Promotion   
1        12346  Wealth planning promotion   
2        12347      Credit Card Promotion   
3        12348  Wealth planning promotion   
4        12349  Wealth planning promotion   

                                       template_text App ID/App Name platform  \
0  Cushion the impact of market fluctuatiions by ...         XYZ app   Mobile   
1  Your finances, our expertise. XYZ bank offers ...         XYZ app   Mobile   
2  Join hands with XYZ as we offer you unique co-...         XYZ app   Mobile   
3  Embrace the future of banking with our new Wea...         XYZ app   Mobile   
4  Stay ahead of financial market trends. Join ou...         XYZ app   Mobile   

                      url                         subject  \
0  https://xyz.com/invest  Enjoy portfolio promotion now!   
1  https://xyz.com/wealth   Personalized Financial Advice   
2   https://xyz.com/cards                   Speical Offer   
3  https://xyz.com/wealth    Unlock Your Wealth Potential   
4  https://xyz.com/wealth    Boost Your Investment Skills   

                                        requirements  
0  Customers between 30 to 80 years old; Minimum ...  
1  Customers between 30 to 80 years old; Minimum ...  
2  Customers between 18 to 80 years old; Minimum ...  
3  Customers between 30 to 80 years old; Minimum ...  
4  Customers between 30 to 80 years old; Minimum ...

## Setup RAG backend - only for one time

In [12]:
# password = input("Enter a password for the cluster: ")

CLUSTER = "cl5"  # @param {type:"string"}
INSTANCE = "rag5"  # @param {type:"string"}

# Set the active Google Cloud Project
!gcloud config set project {PROJECT_ID}

# # Create cluster
# !gcloud alloydb clusters create {CLUSTER} --password={password} --region={REGION}

# # Create the primary instance
# !gcloud alloydb instances create {INSTANCE} --instance-type=PRIMARY --cpu-count={CPU_COUNT} --region={REGION} --cluster={CLUSTER}

# Update the instance to allow public IP
!gcloud beta alloydb instances update {INSTANCE} \
    --region={REGION} \
    --cluster={CLUSTER} \
    --database-flags=password.enforce_complexity="on",password.enforce_password_does_not_contain_username="on" \
    --assign-inbound-public-ip=ASSIGN_IPV4

Updated property [core/project].
Operation ID: operation-1718622505065-61b13fbeadd65-fe234701-226c6ffe


In [13]:
# enable GCP services
!gcloud services enable alloydb.googleapis.com aiplatform.googleapis.com

Operation "operations/acat.p2-3982497274-99ad4d2b-5ae6-4ad3-8092-9af16cb2c50e" finished successfully.


In [14]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
  --member={SERVICE_ACCOUNT} \
  --role="roles/aiplatform.user"

Updated IAM policy for project [rag-project-425902].
bindings:
- members:
  - serviceAccount:service-3982497274@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:3982497274-compute@developer.gserviceaccount.com
  - serviceAccount:service-3982497274@gcp-sa-alloydb.iam.gserviceaccount.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:3982497274-compute@developer.gserviceaccount.com
  role: roles/alloydb.admin
- members:
  - serviceAccount:3982497274-compute@developer.gserviceaccount.com
  role: roles/alloydb.databaseUser
- members:
  - serviceAccount:service-3982497274@gcp-sa-alloydb.iam.gserviceaccount.com
  role: roles/alloydb.serviceAgent
- members:
  - serviceAccount:3982497274@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-3982497274@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service

## Create helper functions

In [15]:
def create_sqlalchemy_engine(
    inst_uri: str, user: str, password: str, db: str
) -> Tuple[sqlalchemy.engine.Engine, Connector]:
    """Creates a connection pool for an AlloyDB instance and returns the pool
    and the connector. Callers are responsible for closing the pool and the
    connector.


    Args:
        inst_uri (str):
            The instance URI specifies the instance relative to the project,
            region, and cluster. For example:
            "projects/my-project/locations/us-central1/clusters/my-cluster/instances/my-instance"
        user (str):
            The database user name, e.g., postgres
        password (str):
            The database user's password, e.g., secret-password
        db (str):
            The name of the database, e.g., mydb

     Returns:
        Tuple[sqlalchemy.engine.Engine, Connector]:
            * A SQLAlchemy engine object for managing database interactions.
            * A Connector object for underlying database connections (can be used for closing).
    """
    connector = Connector()

    def getconn() -> pg8000.dbapi.Connection:
        """
        Establishes a connection to a Google Cloud AlloyDB instance (PostgreSQL database) using the pg8000 driver.

        Returns:
            pg8000.dbapi.Connection: An active database connection object.
        """
        conn: pg8000.dbapi.Connection = connector.connect(
            instance_uri=inst_uri,
            driver="pg8000",
            user=user,
            password=password,
            db=db,
            ip_type="PUBLIC",  # use ip_type to specify Public IP
        )
        return conn

    # create SQLAlchemy connection pool
    engine = sqlalchemy.create_engine(
        "postgresql+pg8000://", creator=getconn, isolation_level="AUTOCOMMIT"
    )
    engine.dialect.description_encoding = None
    return engine, connector
     

In [16]:
def check_table_exists(engine: Engine, connector: Connector, table_name: str) -> bool:
    """Checks if a table exists in the database.

    Args:
        engine (sqlalchemy.engine.Engine): SQLAlchemy engine object.
        connector (Connector): AlloyDB Connector object.
        table_name (str): Name of the table to check.

    Returns:
        bool: True if the table exists, False otherwise.
    """

    try:
        with engine.connect() as conn:
            check_cmd = sqlalchemy.text(f"SELECT 1 FROM {table_name} LIMIT 1")
            conn.execute(check_cmd)
        connector.close()
        return True

    except DatabaseError:
        return False

## Create connection to AlloyDB

In [17]:
INSTANCE_URI = (
    f"projects/{PROJECT_ID}/locations/{REGION}/clusters/{CLUSTER}/instances/{INSTANCE}"
)
USER = "postgres"
DB = "postgres"
TABLE_NAME = "marketing_content"

## Create table on AlloyDB

In [18]:
password='tTt123test!'

In [19]:
engine, connector = create_sqlalchemy_engine(
    inst_uri=INSTANCE_URI,
    user=USER,
    password=password,
    db=DB,
)

In [20]:

if check_table_exists(engine, connector, TABLE_NAME):
    print(f"Table {TABLE_NAME} already exists!")

else:
    # Create table
    create_table_cmd = sqlalchemy.text(
        f"CREATE TABLE {TABLE_NAME} ( \
      template_id VARCHAR, \
      description TEXT, \
      template_text TEXT, \
      url TEXT, \
      platform VARCHAR, \
      subject TEXT, \
      requirements TEXT \
      )",
    )

    # Insert data
    insert_data_cmd = sqlalchemy.text(
        f"""
      INSERT INTO {TABLE_NAME} VALUES (:template_id, :description, :template_text, :url, :platform, :subject, :requirements)
      """
    )

    parameter_map = [
        {
            "template_id": row["template_id"],
            "description": row["description"],
            "template_text": row["template_text"],
            "url": row["url"],
            "platform": row["platform"],
            "subject": row["subject"],
            "requirements": row["requirements"]
        }
        for index, row in df.iterrows()
    ]

    # Execute the queries
    with engine.connect() as conn:
        print("Creating table...")
        conn.execute(create_table_cmd)
        print("Inserting values...")
        conn.execute(
            insert_data_cmd,
            parameter_map,
        )
        print("Committing...")
        conn.commit()
        print("Done")
    connector.close()

Creating table...
Inserting values...
Committing...
Done


## Add alloyDB extensions
- google_ml_integration integrates AlloyDB with Vertex AI
- vector allows us to use pgvector functions and operators with optimizations specific to AlloyDB

In [21]:
# Add extensions
google_ml_integration_cmd = sqlalchemy.text(
    "CREATE EXTENSION IF NOT EXISTS google_ml_integration CASCADE"
)
vector_cmd = sqlalchemy.text("CREATE EXTENSION IF NOT EXISTS vector")

# Execute the queries
with engine.connect() as conn:
    conn.execute(google_ml_integration_cmd)
    conn.execute(vector_cmd)
    conn.commit()
connector.close()
     

## Create a column that stores text embeddings and an Index using AlloyDB
- The Vertex AI text-embeddings API lets you create a text embedding using Generative AI on Vertex AI. Text embeddings are numerical representations of text that capture relationships between words and phrases.
- IVFFlat is a type of vector index for approximate nearest neighbor search. It is a frequently used index type that can improve performance when querying highly-dimensional vectors, like those representing embeddings.

In [22]:
embedding_column = "embedding"
distance_function = "vector_cosine_ops"

# Add column to store embeddings
add_column_cmd = sqlalchemy.text(
    f"ALTER TABLE {TABLE_NAME} ADD COLUMN {embedding_column} vector({DIMENSIONS});"
)

# Generate embeddings for `title` and `abstract` columns of the dataset
embedding_cmd = sqlalchemy.text(
    f"UPDATE {TABLE_NAME} SET {embedding_column} = embedding('{EMBEDDING_MODEL}', subject || ' ' || template_text || ' ' || requirements);"
)

# Create an ivfflat index on the table with embedding column and cosine distance
index_cmd = sqlalchemy.text(
    f"CREATE INDEX ON {TABLE_NAME} USING ivfflat ({embedding_column} {distance_function})"
)

In [23]:
# Execute the queries
with engine.connect() as conn:
    try:
        conn.execute(add_column_cmd)
    except:
        print(f"Column {embedding_column} already exists")
    print("Creating Embeddings...")
    conn.execute(embedding_cmd)
    print("Creating Index...")
    conn.execute(index_cmd)
    print("Committing...")
    conn.commit()
    print("Done")
connector.close()

Creating Embeddings...
Creating Index...
Committing...
Done


## Retrieve data
- based on similarity search

In [24]:
def retrieve_information(
    query: str,
    engine: Engine,
    table_name: str,
    embedding_model: str,
    row_count: int = 5,
) -> str:
    """
    Queries a database table using a semantic similarity search and returns formatted results.

    Args:
        query (str): The search query to embed and compare against the database.
        engine (sqlalchemy.engine.Engine): SQLAlchemy engine object.
        table_name (str): The name of the table to query.
        embedding_model (str): The name of the embedding model to use.
        row_count (int, optional): The maximum number of results to return. Defaults to 5.

    Assumptions:
        The table has columns named 'template_id', 'subject', 'template_text', description', 'url', platform', 'requirements', and an embedding column named 'embedding_column'.

    Returns:
        str: A formatted string containing the top results, including their publication number, title, abstract, and URL.
    """

    # Perform semantic search
    search_cmd = sqlalchemy.text(
        f"""
    SELECT template_id, subject, template_text, description, platform, requirements FROM {table_name}
      ORDER BY  {embedding_column}
      <-> embedding('{embedding_model}', '{query}')::vector
      LIMIT {row_count}
    """
    )

    # Execute the query
    with engine.connect() as conn:
        result = conn.execute(search_cmd)
        context = [row._asdict() for row in result]
    connector.close()

    # String format the retrieved information
    retrieved_information = "\n".join(
        [
            f"{index+1}. "
            + "\n".join([f"{key}: {value}" for key, value in element.items()])
            for index, element in enumerate(context)
        ]
    )

    return retrieved_information

In [25]:
def generate_text(
    use_rag: True,
    prompt: str,
    result: str,
    question: str,
    generative_model: GenerativeModel,
    generation_config: GenerationConfig,
    model: non_rag_model
) -> str:
    if use_rag:
        """
        Generates text response using a specified generative language model on Vertex AI, with RAG.

        Args:
            prompt (str): The text prompt template for the generative model.
            result (str): The list of matching patents.
            question (str): The user's question.
            generative_model (vertexai.generative_models.GenerativeModel): The name or identifier of the generative model on Vertex AI.
            generation_config (vertexai.generative_models.GenerationConfig): Configuration object for the text generation process.

        Returns:
            str: The generated text response from the model.
        """
        input_prompt = prompt.format(result=result, question=question)
        print('**Prompt (with RAG)**\n\n', input_prompt, '**End of Prompt**')
        
        # Query the model
        response = generative_model.generate_content(
            contents=input_prompt, generation_config=generation_config
        )

        return response.text
    
    else:
        # Without RAG
        
        contents = [prompt]
        # Prompt the model to generate content
        print('**Prompt (without RAG)**\n\n', prompt, '**End of Prompt**')

        response = model.generate_content(
            contents,
            generation_config=generation_config,
            safety_settings=safety_settings)
        
        return response.candidates[0].content.parts[0].text
        

### Sample Questions

- Propose some ideas for the query entered

In [26]:
query = "Come up with ideas on remittance promotions."  # @param {type:"string"}

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)
print(result)

1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
2. template_id: 12376
subject: Fulfill your global needs when you remit with XYZ
template_text: Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, 

# Prompting with & without RAG

## 1. Zero-shot prompting
- Zero-shot prompting relies on the model's ability to understand the task without any examples. The prompt contains only the instruction or question without any prior examples.

### Messages

In [168]:
query = "Generate a remittance promotion message from Singapore to Malaysia"  # @param {type:"string"}

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

zs_prompt = """
Use the following pieces of context to answer the below question/request. 
{question}

{result}
"""

print(result)

1. template_id: 12394
subject: <ADV> Send money from Singapore to Malaysia at preferential rates!
template_text: From now till 30 April 2024, enjoy preferential exchange rates and avoid cable/commission fees when you transfer funds from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) accounts. Rest assured that rates quoted on our website are guaranteed even during market off hours (including public holidays and weekends). Log in to XYZ Internet Banking or the XYZ Digital app to get started now! Terms and conditions apply.
description: Exchange Rate Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$3000
2. template_id: 12370
subject: Send money to Malaysia at preferential rates. No cable/commission fees charged!
template_text: Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/c

- With RAG

In [242]:
zs_response = generate_text(
    use_rag=True,
    prompt=zs_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(zs_response)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 

Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.


1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visi

- without RAG

In [245]:
zs_prompt = """
Use the following pieces of context to answer the below question/request. 
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.

{result}
"""

zs_response_no_rag = generate_text(
    use_rag=False,
    prompt=zs_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(zs_response_no_rag)

**Prompt (without RAG)**

 
Use the following pieces of context to answer the below question/request. 
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.

{result}
 **End of Prompt**
## Subject: Invest Your Bonus Wisely with XYZ Bank and Secure Your Financial Future

Dear [Customer Name],

Congratulations on receiving your annual bonus! This is an excellent opportunity to invest in your future and achieve your financial goals. At XYZ Bank, we understand the importance of making smart investment decisions, especially when it comes to securing your long-term financial security.

We offer a comprehensive range of investment services tailored to meet the individual needs of our clients. Whether you're looking for a conservative approach with guaranteed returns or a more aggressive strategy with the potential for higher growth

### Emails

- with RAG

In [246]:
query = "Write an email for XYZ Bank to promote their investment services, highlighting the advantages over competitors"  # @param {type:"string"}

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

zs_prompt2 = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

In [247]:
zs_response2 = generate_text(
    use_rag=True,
    prompt=zs_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(zs_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit 

- without RAG

In [248]:
zs_prompt2 = """
Write an email for XYZ Bank to promote their investment services, highlighting the advantages over competitors
{result}
"""

zs_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=zs_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(zs_response_no_rag2)

**Prompt (without RAG)**

 
Write an email for XYZ Bank to promote their investment services, highlighting the advantages over competitors
{result}
 **End of Prompt**
## Subject: Invest with Confidence: Why XYZ Bank is Your Best Choice

Dear [Customer Name],

Are you looking to grow your wealth and secure your financial future? Look no further than XYZ Bank's comprehensive investment services. We offer a wide range of solutions tailored to your individual needs and goals, ensuring you achieve maximum returns while minimizing risk.

**Here's why XYZ Bank stands out from the competition:**

* **Experienced and Dedicated Team:** Our team of financial experts possesses years of experience and in-depth knowledge of the market, allowing them to provide personalized guidance and support throughout your investment journey.
* **Diversified Investment Options:** We offer a vast array of investment options, including stocks, bonds, mutual funds, ETFs, and alternative investments. This allows you 

## 2. Direct prompting
- This involves providing a straightforward instruction or question to the model. The prompt is clear and unambiguous, guiding the model to produce the expected output.

### Messages

- with RAG

In [174]:
query = "Generate a remittance promotion message with the subject: XYZ Remit: Your Gateway to China"  # @param {type:"string"}

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

direct_prompt = """
Use the following pieces of context to answer the below question/request. 
{question}
    **Example:**
    - Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more."
{result}
"""

In [175]:
direct_response = generate_text(
    use_rag=True,
    prompt=direct_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(direct_response)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
Generate a remittance promotion message with the subject: XYZ Remit: Your Gateway to China
    **Example:**
    - Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more."
1. template_id: 12353
subject: XYZ Remit: Your Gateway to China
template_text: Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.
description: Remittance promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees 

In [176]:
direct_prompt = """
Generate a remittance promotion message with the subject: XYZ Remit: Your Gateway to China
    **Example:**
    - Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more."
{result}
"""

direct_response_no_rag = generate_text(
    use_rag=False,
    prompt=direct_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(direct_response_no_rag)

**Prompt (without RAG)**

 
Generate a remittance promotion message with the subject: XYZ Remit: Your Gateway to China
    **Example:**
    - Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more."
{result}
 **End of Prompt**
## Subject: XYZ Remit: Your Gateway to China

## Body:

Hey there,

Looking to send money to China? Look no further than XYZ Remit! 

**Here's why you should choose us:**

* **Competitive rates:** Get the most out of your money with our low exchange rates.
* **$0 transfer fees:** Save even more with our zero transfer fees on all transactions to China.
* **Fast and secure:** Your money will be delivered to your loved ones quickly and securely.
* **Easy to use:** Our platform is simple and straightforward, so you can send money in minutes.

**Ready to get started?**

Click here to learn more and send your first transfer today!

**Limited-time offer:**

For a limited time, new users can enjoy 

### Emails

- with RAG

In [177]:
query = "Compose an email for XYZ Bank that encourages recipients to explore their investment options."  # @param {type:"string"}

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

direct_prompt2 = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

In [178]:
direct_response2 = generate_text(
    use_rag=True,
    prompt=direct_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(direct_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
Compose an email for XYZ Bank that encourages recipients to explore their investment options.
1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit www.xyz.com to get started.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 30 to 80 

- without RAG

In [179]:
direct_prompt2 = """
Compose an email for XYZ Bank that encourages recipients to explore their investment options.
{result}
"""

direct_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=direct_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(direct_response_no_rag2)

**Prompt (without RAG)**

 
Compose an email for XYZ Bank that encourages recipients to explore their investment options.
{result}
 **End of Prompt**
## Subject: Unlock Your Financial Future with XYZ Bank's Investment Solutions

Dear Valued Customer,

Are you looking for ways to grow your wealth and secure your financial future? Look no further than XYZ Bank! We offer a wide range of investment options to meet your individual needs and goals.

**Why Invest with XYZ Bank?**

* **Experienced and knowledgeable team:** Our team of financial experts has years of experience in guiding clients towards successful investments. 
* **Diverse investment options:** We offer a variety of investment products, including stocks, bonds, mutual funds, ETFs, and more. 
* **Personalized investment plans:** We work closely with you to create a customized investment plan that aligns with your risk tolerance and financial objectives.
* **Competitive rates and fees:** We offer competitive rates and fees to hel

## 3. Few shot prompting
- In few-shot prompting, the model is given a few examples of the task it is supposed to perform before the actual prompt. This helps the model understand the task better and produce more accurate results.

### Messages

- with RAG

In [191]:
query = """Generate remittance promotion messages for the following subjects: 
1. XYZ Remit: Your Gateway to China-> 
Enjoy competitive rates and $0 transfer fees when you send money to China 
with XYZ Remit! Tap to find out more.
2. Send money to Malaysia at preferential rates. No cable/commission fees charged!
-> Make your money go further. Enjoy preferential FX rates when 
you make online transfers from your XYZ Singapore (SGD) account 
to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. 
Save on fees when you send money home. Fulfill your global needs when you remit with XYZ

"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

fs_prompt = """
Use the following pieces of context to answer the below question/request. 
{question}
**Example:**
- Expected Output: "Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply. Remit now"
{result}
"""

In [192]:
fs_response = generate_text(
    use_rag=True,
    prompt=fs_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(fs_response)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
Generate remittance promotion messages for the following subjects: 
1. XYZ Remit: Your Gateway to China-> 
Enjoy competitive rates and $0 transfer fees when you send money to China 
with XYZ Remit! Tap to find out more.
2. Send money to Malaysia at preferential rates. No cable/commission fees charged!
-> Make your money go further. Enjoy preferential FX rates when 
you make online transfers from your XYZ Singapore (SGD) account 
to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. 
Save on fees when you send money home. Fulfill your global needs when you remit with XYZ


**Example:**
- Expected Output: "Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or prope

- without RAG

In [193]:
fs_prompt = """
Generate remittance promotion messages for the following subjects: XYZ Remit: Your Gateway to China ' -> Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.' ‘Send money to Malaysia at preferential rates. No cable/commission fees charged!' -> Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. 'Fulfill your global needs when you remit with XYZ' ->"
- Expected Output: " Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply. Remit now
{result}
"""

fs_response_no_rag = generate_text(
    use_rag=False,
    prompt=fs_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(fs_response_no_rag)

**Prompt (without RAG)**

 
Generate remittance promotion messages for the following subjects: XYZ Remit: Your Gateway to China ' -> Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.' ‘Send money to Malaysia at preferential rates. No cable/commission fees charged!' -> Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. 'Fulfill your global needs when you remit with XYZ' ->"
- Expected Output: " Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 D

### Emails

- with RAG

In [194]:
query = """Given these examples of email marketing messages for bank services, 
write a new one for XYZ Bank’s investment services focusing 
on risk management and potential returns. 

Examples: 
#1. Cushion the impact of market fluctuatiions 
by investing a fixed amount regularly! Simply set up a 
recurring top up and dollar-cost average with xyzPortfolio 
today!

#2. Seeking hassle-free invsesting? Tap here to select 
your preferred review frequency, and we will tailor product recommendations 
to match your preference.

#3. Limited tranche! Get potential 
monthly dividends and capital growth. Invest in XYZ new 
globally diversified fund. This advertisement has not been 
reviewed by the Monetary Authority of Singapore. Investments 
are subject to investment risks, including the possible loss 
of the principal amount invested. Explore this option before 
the tranche is fully subscribed.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

fs_prompt2 = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

In [195]:
fs_response2 = generate_text(
    use_rag=True,
    prompt=fs_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(fs_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
Given these examples of email marketing messages for bank services, 
write a new one for XYZ Bank’s investment services focusing 
on risk management and potential returns. 

Examples: 
#1. Cushion the impact of market fluctuatiions 
by investing a fixed amount regularly! Simply set up a 
recurring top up and dollar-cost average with xyzPortfolio 
today!

#2. Seeking hassle-free invsesting? Tap here to select 
your preferred review frequency, and we will tailor product recommendations 
to match your preference.

#3. Limited tranche! Get potential 
monthly dividends and capital growth. Invest in XYZ new 
globally diversified fund. This advertisement has not been 
reviewed by the Monetary Authority of Singapore. Investments 
are subject to investment risks, including the possible loss 
of the principal amount invested. Explore this option before 
the tranche is fully subscribed.

1. templat

- without RAG

In [196]:
fs_prompt2 = """
Given these examples of email marketing messages for bank services, 
write a new one for XYZ Bank’s investment services focusing 
on risk management and potential returns. 

Examples: 
#1. Cushion the impact of market fluctuatiions 
by investing a fixed amount regularly! Simply set up a 
recurring top up and dollar-cost average with xyzPortfolio 
today!

#2. Seeking hassle-free invsesting? Tap here to select 
your preferred review frequency, and we will tailor product recommendations 
to match your preference.

#3. Limited tranche! Get potential 
monthly dividends and capital growth. Invest in XYZ new 
globally diversified fund. This advertisement has not been 
reviewed by the Monetary Authority of Singapore. Investments 
are subject to investment risks, including the possible loss 
of the principal amount invested. Explore this option before 
the tranche is fully subscribed.
{result}
"""

fs_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=fs_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(fs_response_no_rag2)

**Prompt (without RAG)**

 
Given these examples of email marketing messages for bank services, 
write a new one for XYZ Bank’s investment services focusing 
on risk management and potential returns. 

Examples: 
#1. Cushion the impact of market fluctuatiions 
by investing a fixed amount regularly! Simply set up a 
recurring top up and dollar-cost average with xyzPortfolio 
today!

#2. Seeking hassle-free invsesting? Tap here to select 
your preferred review frequency, and we will tailor product recommendations 
to match your preference.

#3. Limited tranche! Get potential 
monthly dividends and capital growth. Invest in XYZ new 
globally diversified fund. This advertisement has not been 
reviewed by the Monetary Authority of Singapore. Investments 
are subject to investment risks, including the possible loss 
of the principal amount invested. Explore this option before 
the tranche is fully subscribed.
{result}
 **End of Prompt**
## Subject: Invest with Confidence: Manage Risk and Max

## 4. Chain-of-Thought prompting
- This technique encourages the model to "think aloud" and break down the reasoning process step by step, leading to the final answer. It's useful for tasks requiring logical reasoning or multi-step problem solving.

### Messages

- with RAG

In [197]:
query = """Craft a promotional message for the XYZs remittance service emphasizing its exclusive promotion. Highlight the service low charges and extensive countries coverage."""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)


cot_prompt = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: {question}

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.
{result}
"""

- with RAG

In [198]:
cot_response = generate_text(
    use_rag=True,
    prompt=cot_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=4096),
    model=non_rag_model
)

print(cot_response)

**Prompt (with RAG)**

 
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audien

- without RAG

In [253]:
cot_prompt = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: Craft a promotional message for the XYZs remittance service emphasizing its exclusive promotion. Highlight the service low charges and extensive countries coverage.

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.
{result}
"""

In [254]:
cot_response_no_rag = generate_text(
    use_rag=False,
    prompt=cot_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(cot_response_no_rag)

**Prompt (without RAG)**

 
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target aud

### Emails

- with RAG

In [200]:
query = """Craft a promotional message for the XYZs wealth planning services. Highlight its benefits and the high returns that the service is able to provide."""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)


cot_prompt2 = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: {question}

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.
{result}
"""

In [202]:
cot_response2 = generate_text(
    use_rag=True,
    prompt=cot_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=4096),
    model=non_rag_model
)

print(cot_response2)

**Prompt (with RAG)**

 
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audien

- without rag

In [255]:
query = """Craft a promotional message for the XYZs wealth planning services. Highlight its benefits and the high returns that the service is able to provide."""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)


cot_prompt2 = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: Craft a promotional message for the XYZs wealth planning services. Highlight its benefits and the high returns that the service is able to provide.

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.
{result}
"""

In [256]:
cot_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=cot_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(cot_response_no_rag2)

**Prompt (without RAG)**

 
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target aud

## 5. Contextual Prompting
- In contextual prompting, the model is given a context or background information before the actual prompt. This context helps the model understand the situation or topic better.

### Messages

- with RAG

In [204]:
query = """Context: "Bank XYZ is offering excellent FX rates for online transfers between Singapore and Malaysia accounts, from SGD to MYR. 
Commission fees will also be waived for these transfers. 
The promotion is subjected to T&Cs.

Generate a promotional message for cross border transfers with Bank XYZ.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

contextual_prompt = """
Use the following pieces of context to answer the below question/request. 
{question}
- Expected Output: "Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. "
{result}
"""

In [205]:
contextual_response = generate_text(
    use_rag=True,
    prompt=contextual_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model

)

print(contextual_response)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
Context: "Bank XYZ is offering excellent FX rates for online transfers between Singapore and Malaysia accounts, from SGD to MYR. 
Commission fees will also be waived for these transfers. 
The promotion is subjected to T&Cs.

Generate a promotional message for cross border transfers with Bank XYZ.

- Expected Output: "Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. "
1. template_id: 12394
subject: <ADV> Send money from Singapore to Malaysia at preferential rates!
template_text: From now till 30 April 2024, enjoy preferential exchange rates and avoid cable/commission fees when you transfer funds from your XYZ Singapore (SGD) account to a

- without RAG

In [206]:
contextual_prompt = """
**Example:**
- Context: "Bank XYZ is offering excellent FX rates for online transfers between Singapore and Malaysia accounts, from SGD to MYR. Commission fees will also be waived for these transfers. The promotion is subjected to T&Cs."
- Prompt: "Generate a promotional message for cross border transfers with Bank XYZ."
- Expected Output: "Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. "
{result}
"""

contextual_response_no_rag = generate_text(
    use_rag=False,
    prompt=contextual_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model

)

print(contextual_response_no_rag)

**Prompt (without RAG)**

 
**Example:**
- Context: "Bank XYZ is offering excellent FX rates for online transfers between Singapore and Malaysia accounts, from SGD to MYR. Commission fees will also be waived for these transfers. The promotion is subjected to T&Cs."
- Prompt: "Generate a promotional message for cross border transfers with Bank XYZ."
- Expected Output: "Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. "
{result}
 **End of Prompt**
## Example 1:

**Context:** "A new report by the World Bank has found that the global economy is expected to grow by 3% in 2023. This is a slight downgrade from the previous forecast of 3.2%. The report cites ongoing geopolitical tensions and rising inflation as the main reasons for the downgr

### Emails
- with RAG

In [207]:
query = """Context: "Bank XYZ is offering wealth planning services. It helps to promote tax savings and users can be worry-free
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

contextual_prompt2 = """
Use the following pieces of context to answer the below question/request. 
{question}
Generate an email message with the above information.
{result}
"""

In [208]:
contextual_response2 = generate_text(
    use_rag=True,
    prompt=contextual_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model

)

print(contextual_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
Context: "Bank XYZ is offering wealth planning services. It helps to promote tax savings and users can be worry-free

Generate an email message with the above information.
1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12352
subject: Retirement Planning Services
template_text: Dear Alice, Prepare for the future with XYZ Bank's professional retirement planning services to help you achieve financial freedom. Our retirement planning advisors will work with you one-on-one to ensure a worry-free retirement life. The XYZ Bank Team
descr

- without RAG

In [257]:
contextual_prompt2 = """
Use the following pieces of context to answer the below question/request. 
Context: "Bank XYZ is offering wealth planning services. It helps to promote tax savings and users can be worry-free
Generate an email message with the above information.
{result}
"""

contextual_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=contextual_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model

)

print(contextual_response_no_rag2)

**Prompt (without RAG)**

 
Use the following pieces of context to answer the below question/request. 
Context: "Bank XYZ is offering wealth planning services. It helps to promote tax savings and users can be worry-free
Generate an email message with the above information.
{result}
 **End of Prompt**
## Subject: Secure Your Future with Bank XYZ's Wealth Planning Services

Dear [Customer Name],

Are you looking for ways to save on taxes and secure your financial future? Look no further than Bank XYZ's comprehensive wealth planning services!

At Bank XYZ, we understand that navigating the complexities of financial planning can be overwhelming. That's why we offer personalized wealth planning solutions tailored to your unique needs and goals. Our team of experienced professionals will work closely with you to develop a customized strategy that helps you:

* **Minimize your tax liability:** We'll identify potential tax-saving opportunities and recommend strategies to help you reduce your t

## 6. Role-playing or Persona-based prompt
- This technique involves assigning a specific role or persona to the model, influencing the style and content of the response.

### Messages

In [210]:
query = """You are a marketing agent for Bank XYZ. 
Generate a message to promote the bank’s remittance product which 
offers no transfer fees to China.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

persona_prompt = """
Use the following pieces of context to answer the below question/request. 
{question}
- Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.”
{result}
"""

- with RAG

In [211]:
persona_response = generate_text(
    use_rag=True,
    prompt=persona_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(persona_response)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
You are a marketing agent for Bank XYZ. 
Generate a message to promote the bank’s remittance product which 
offers no transfer fees to China.

- Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.”
1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirement

- without RAG

In [212]:
persona_prompt = """
**Example:**
- Prompt: "You are a marketing agent for Bank XYZ. Generate a message to promote the bank’s remittance product which offers no transfer fees to China."
- Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.”
{result}
"""

persona_response_no_rag = generate_text(
    use_rag=False,
    prompt=persona_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(persona_response_no_rag)

**Prompt (without RAG)**

 
**Example:**
- Prompt: "You are a marketing agent for Bank XYZ. Generate a message to promote the bank’s remittance product which offers no transfer fees to China."
- Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.”
{result}
 **End of Prompt**
## 

## Bank XYZ Remit: Send Money to China with Zero Fees!

Tired of high transfer fees when sending money to China? Look no further than Bank XYZ Remit! 

**With Bank XYZ Remit, you can enjoy:**

* **$0 transfer fees to China**
* **Competitive exchange rates**
* **Fast and secure transactions**
* **Easy-to-use online platform**

**Whether you're supporting loved ones, paying bills, or investing in China, Bank XYZ Remit makes it easy and affordable.**

**Get started today and experience the difference!**

**Click here to learn more.**

**#BankXYZRemit #SendMoneyToChina #NoTransferFees** 



### Emails
- with RAG

In [213]:
query = """As a young professional looking to secure your 
financial future, XYZ Bank’s investment options offer the 
perfect balance of risk and reward. Craft an email targeting 
this persona.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

persona_prompt2 = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

In [214]:
persona_response2 = generate_text(
    use_rag=True,
    prompt=persona_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(persona_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 
As a young professional looking to secure your 
financial future, XYZ Bank’s investment options offer the 
perfect balance of risk and reward. Craft an email targeting 
this persona.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12441
subject: <ADV> Your guide to financial planning in your 20s
template_text: Dear Valued Client,
Being in your 20s means time is on your side. While you've just stepped into the adulting world, take advantage of the magic of compounding and watch your money grow over time. Unleash your full financial

- without RAG

In [216]:
persona_prompt2 = """
As a young professional looking to secure your financial future, XYZ Bank’s investment options offer the perfect balance of risk and reward. Craft an email targeting this persona.
{result}
"""

persona_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=persona_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(persona_response_no_rag2)

**Prompt (without RAG)**

 
As a young professional looking to secure your financial future, XYZ Bank’s investment options offer the perfect balance of risk and reward. Craft an email targeting this persona.
{result}
 **End of Prompt**
## Subject: Secure Your Financial Future with XYZ Bank's Investment Options

Dear [Young Professional Name],

As a young professional, you're likely starting to think about your financial future. You're ambitious, driven, and want to ensure your hard work translates into long-term success. At XYZ Bank, we understand your aspirations and are here to help you achieve them.

We offer a diverse range of investment options designed to meet the needs of young professionals like yourself. Whether you're looking for steady growth, income generation, or a combination of both, our experienced financial advisors will work with you to create a personalized investment plan that aligns with your risk tolerance and financial goals.

**Here's what sets XYZ Bank apart:**

## 7. Completion prompt
- Here, the model is given an incomplete sentence or story and asked to complete it. This technique is often used in creative writing and story generation.

### Messages

In [217]:
query = """Make your money go further. Enjoy preferential FX rates …
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

completion_prompt = """
Use the following pieces of context to complete the response. 
{question}
- Expected Output: " when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home."
{result}
"""


- With RAG

In [219]:
completion_response = generate_text(
    use_rag=True,
    prompt=completion_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(completion_response)

**Prompt (with RAG)**

 
Use the following pieces of context to complete the response. 
Make your money go further. Enjoy preferential FX rates …

- Expected Output: " when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home."
1. template_id: 12423
subject: <ADV> Limited Time: Enjoy ZERO foreign exchange (FX) fees when you spend overseas with XYZ Credit Cards
template_text: What better way to travel – enjoy ZERO FX fees on your overseas spends (min. spend of S$300 equivalent) with any XYZ Credit Cards in Malaysia, Thailand and Japan from 15 March to 15 April 2024. 
To register, simply SMS SCFX to 77222.
Please visit our official website for offer details. Terms and Conditions apply.
description: Remittance promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly

- without RAG

In [220]:
completion_prompt = """
**Example:**
- Prompt: " Make your money go further. Enjoy preferential FX rates …"
- Expected Output: " when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home."
{result}
"""

completion_response_no_rag = generate_text(
    use_rag=False,
    prompt=completion_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(completion_response_no_rag)

**Prompt (without RAG)**

 
**Example:**
- Prompt: " Make your money go further. Enjoy preferential FX rates …"
- Expected Output: " when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home."
{result}
 **End of Prompt**
- Prompt: "I have a 2015 Honda Civic EX-T with 100,000 miles on it. I'm looking to get a new car, but I'm not sure what to get. I like the size and fuel efficiency of the Civic, but I'm also interested in something with more features and technology. I'm open to both new and used cars, and my budget is around $25,000. What are some good options for me?"
- Expected Output: "## Options for Replacing Your 2015 Honda Civic EX-T

With your 2015 Honda Civic EX-T nearing 100,000 miles and a desire for newer features and technology, it's time to explore your options for a replacement. Here a

### Emails
- with RAG

In [221]:
query = """XYZ Bank’s new investment plan is designed to offer the best returns on the market. This plan includes...
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

completion_prompt2 = """
Use the following pieces of context to complete the response. 
{question}

Complete the request above with a promotional email message.
{result}
"""


In [222]:
completion_response2 = generate_text(
    use_rag=True,
    prompt=completion_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(completion_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to complete the response. 
XYZ Bank’s new investment plan is designed to offer the best returns on the market. This plan includes...


Complete the request above with a promotional email message.
1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12400
subject: <ADV> Earn bigger returns with up to 4.65% a year when you credit your salary, save and spend on an XYZ Account
template_text: From up to 4.05% a year, we’ve increased our interest rate to up to 4.65% when you credit your salary, save and spend on your XYZ Account. Earn bigger, better gains of additional 3% a year w

- without RAG

In [223]:
completion_prompt2 = """
XYZ Bank’s new investment plan is designed to offer the best returns on the market. This plan includes...
{result}
"""

completion_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=completion_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(completion_response_no_rag2)

**Prompt (without RAG)**

 
XYZ Bank’s new investment plan is designed to offer the best returns on the market. This plan includes...
{result}
 **End of Prompt**
## XYZ Bank's New Investment Plan: 

**XYZ Bank's new investment plan** promises to offer the best returns on the market. This plan includes:

* **High-yield investments:** The plan invests in a variety of high-yield assets, such as stocks, bonds, and real estate, with the potential to generate significant returns.
* **Diversification:** The plan is well-diversified across different asset classes and industries, which helps to mitigate risk and improve the chances of achieving consistent returns.
* **Professional management:** The plan is managed by a team of experienced investment professionals who have a proven track record of success.
* **Low fees:** XYZ Bank offers competitive fees on its investment plans, making it an affordable option for investors of all levels.
* **Flexibility:** Investors can choose from a variety of 

## 8. Guided Prompt
- Providing detailed instructions or Prompting constraints to direct the model’s response in a specific way.

### Messages

In [224]:
query = """
Generate a compelling marketing message for XYZ Bank that highlights the benefits of their new remittance promotions for customers looking to remit money overseas.
Focus on the features and potential rebates customers will get to enjoy.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

guided_prompt = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

- with RAG

In [225]:
guided_response = generate_text(
    use_rag=True,
    prompt=guided_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(guided_response)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 

Generate a compelling marketing message for XYZ Bank that highlights the benefits of their new remittance promotions for customers looking to remit money overseas.
Focus on the features and potential rebates customers will get to enjoy.

1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years o

- without RAG

In [258]:

guided_prompt = """
Use the following pieces of context to answer the below question/request. 
Generate a compelling marketing message for XYZ Bank that highlights the benefits of their new remittance promotions for customers looking to remit money overseas.
Focus on the features and potential rebates customers will get to enjoy.
{result}
"""

In [259]:
guided_prompt_no_rag = """
Generate a compelling marketing message for XYZ Bank that highlights the benefits of their new remittance promotions for customers looking to remit money overseas.
Focus on the features and potential rebates customers will get to enjoy.
{result}
"""

guided_response_no_rag = generate_text(
    use_rag=False,
    prompt=guided_prompt_no_rag,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(guided_response_no_rag)

**Prompt (without RAG)**

 
Generate a compelling marketing message for XYZ Bank that highlights the benefits of their new remittance promotions for customers looking to remit money overseas.
Focus on the features and potential rebates customers will get to enjoy.
{result}
 **End of Prompt**
## Send Money Overseas with Ease and Save Big with XYZ Bank!

**Are you looking to send money internationally?** Look no further than XYZ Bank! We're excited to announce our **limited-time remittance promotion**, offering you incredible benefits and **potential rebates** when you choose us for your overseas transfers.

**Here's what you can enjoy with XYZ Bank:**

* **Competitive exchange rates:** Get the most out of your money with our competitive rates, ensuring you send more to your loved ones abroad.
* **Low transfer fees:** We believe in making international transfers affordable. That's why we offer some of the lowest fees in the market.
* **Fast and secure transactions:** Your money will reac

### Emails

In [261]:
query = """
Generate a compelling email marketing message for XYZ Bank that highlights the benefits of their new investment plan for customers looking to maximize their savings. 
Focus on the features and potential returns of the plan.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

guided_prompt2 = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

- with RAG

In [262]:
guided_response2 = generate_text(
    use_rag=True,
    prompt=guided_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(guided_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 

Generate a compelling email marketing message for XYZ Bank that highlights the benefits of their new investment plan for customers looking to maximize their savings. 
Focus on the features and potential returns of the plan.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12381
subject: Unlock Your Financial Potential with Our New Savings Account!
template_text: Open a new savings account with XYZ today and enjoy competitive interest rates and easy online management. Secure your financial future with us! Visit www.xyz.com or call 

In [260]:
guided_prompt2 = """
Use the following pieces of context to answer the below question/request. 
Generate a compelling email marketing message for XYZ Bank that highlights the benefits of their new investment plan for customers looking to maximize their savings. 
Focus on the features and potential returns of the plan.
{result}
"""

- without RAG

In [263]:
guided_prompt2 = """
Use the following pieces of context to answer the question at the end. 
Generate a compelling email marketing message for XYZ Bank that highlights the benefits of their new investment plan for customers looking to maximize their savings. 
Focus on the features and potential returns of the plan.
{result}
"""

guided_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=guided_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(guided_response_no_rag2)

**Prompt (without RAG)**

 
Use the following pieces of context to answer the question at the end. 
Generate a compelling email marketing message for XYZ Bank that highlights the benefits of their new investment plan for customers looking to maximize their savings. 
Focus on the features and potential returns of the plan.
{result}
 **End of Prompt**
## Subject: Maximize Your Savings with XYZ Bank's New Investment Plan!

**Dear [Customer Name],**

Are you looking for a way to grow your savings and secure your financial future? Look no further than XYZ Bank's new **[Investment Plan Name]**! This exciting plan offers you:

* **High potential returns:** Invest in a diversified portfolio of assets with the potential to significantly outpace traditional savings accounts.
* **Flexibility:** Choose from a variety of investment options to match your risk tolerance and financial goals.
* **Expert guidance:** Our team of experienced financial advisors is here to help you choose the right investme

guided_prompt2 = """
Use the following pieces of context to answer the question at the end. 
Create a short and engaging SMS for XYZ Bank to 
promote their savings account options, emphasizing the 
high interest rates and flexibility offered to new customers.
{result}
"""

guided_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=guided_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(guided_response_no_rag2)

## 9. Hypothetical prompting
- Posing a “what if” scenario or a hypothetical situation to explore potential responses or outcomes.

### Messages

- with RAG

In [230]:
query = """
Imagine a customer looking to remit their money overseas monthly to his/her family after receiving monthly salaries. Write a marketing message for XYZ Bank that convinces them to use XYZ’s remittance services by illustrating the benefits of remitting money monthly with XYZ.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

hypothetical_prompt = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

In [231]:
hypothetical_response = generate_text(
    use_rag=True,
    prompt=hypothetical_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(hypothetical_response)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 

Imagine a customer looking to remit their money overseas monthly to his/her family after receiving monthly salaries. Write a marketing message for XYZ Bank that convinces them to use XYZ’s remittance services by illustrating the benefits of remitting money monthly with XYZ.

1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requiremen

- without RAG

In [264]:
hypothetical_prompt = """
Imagine a customer looking to remit their money overseas monthly to his/her family after receiving monthly salaries. Write a marketing message for XYZ Bank that convinces them to use XYZ’s remittance services by illustrating the benefits of remitting money monthly with XYZ.
{result}
"""

In [265]:
hypothetical_response_no_rag = generate_text(
    use_rag=False,
    prompt=hypothetical_prompt,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(hypothetical_response_no_rag)

**Prompt (without RAG)**

 
Imagine a customer looking to remit their money overseas monthly to his/her family after receiving monthly salaries. Write a marketing message for XYZ Bank that convinces them to use XYZ’s remittance services by illustrating the benefits of remitting money monthly with XYZ.
{result}
 **End of Prompt**
## XYZ Bank: Your Reliable Partner for Monthly Remittances

**Are you sending money overseas to your loved ones every month?** 

At XYZ Bank, we understand the importance of supporting your family back home. That's why we offer a convenient and affordable way to send money internationally on a regular basis.

**Here's why you should choose XYZ Bank for your monthly remittances:**

* **Competitive exchange rates:** We offer some of the best exchange rates in the market, ensuring your family receives the maximum value for your money.
* **Low transfer fees:** We keep our transfer fees low, so you can send more money to your loved ones.
* **Fast and reliable transf

### Emails

- with RAG

In [266]:
query = """
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

hypothetical_prompt2 = """
Use the following pieces of context to answer the below question/request. 
{question}
{result}
"""

In [267]:
hypothetical_response2 = generate_text(
    use_rag=True,
    prompt=hypothetical_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(hypothetical_response2)

**Prompt (with RAG)**

 
Use the following pieces of context to answer the below question/request. 

Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit

query = """
Suppose a customer is unsure about the best way to save 
for their child’s education. Write an SMS for XYZ Bank 
that introduces their high-yield savings account as the 
perfect solution.
"""

result = retrieve_information(
    query=query, engine=engine, table_name=TABLE_NAME, embedding_model=EMBEDDING_MODEL
)

hypothetical_prompt2 = """
Use the following pieces of context to answer the question at the end. 
{question}
{result}
"""

hypothetical_response2 = generate_text(
    use_rag=True,
    prompt=hypothetical_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(hypothetical_response2)

- without RAG

In [269]:
hypothetical_prompt2 = """
Use the following pieces of context to answer the below question/request. 
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.
{result}
"""

In [270]:
hypothetical_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=hypothetical_prompt2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(hypothetical_response_no_rag2)

**Prompt (without RAG)**

 
Use the following pieces of context to answer the below question/request. 
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.
{result}
 **End of Prompt**
## Subject: Invest Your Bonus Wisely with XYZ Bank and Secure Your Financial Future

Dear [Customer Name],

Congratulations on receiving your annual bonus! This is a fantastic opportunity to invest in your future and build a solid foundation for financial security. At XYZ Bank, we understand the importance of making smart investment decisions, and we're here to help you achieve your financial goals.

**Why Choose XYZ Bank for Your Investment Needs?**

* **Personalized Investment Plans:** Our experienced financial advisors will work with you to create a customized investment plan tailored to your specific needs and risk tolerance. 
* **Diversif

hypothetical_prompt_no_rag2 = """
Suppose a customer is unsure about the best way to save 
for their child’s education. Write an SMS for XYZ Bank 
that introduces their high-yield savings account as the 
perfect solution.
{result}
"""

hypothetical_response_no_rag2 = generate_text(
    use_rag=False,
    prompt=hypothetical_prompt_no_rag2,
    result=result,
    question=query,
    generative_model=GenerativeModel(GENERATIVE_MODEL),
    generation_config=GenerationConfig(temperature=0.6, max_output_tokens=1024),
    model=non_rag_model
)

print(hypothetical_response_no_rag2)

## Compile results

In [271]:
dict_results = {
    "prompt":[zs_prompt, zs_prompt2, direct_prompt, direct_prompt2, fs_prompt, fs_prompt2, cot_prompt, cot_prompt2, contextual_prompt, contextual_prompt2, persona_prompt, persona_prompt2, completion_prompt, completion_prompt2, guided_prompt, guided_prompt2, hypothetical_prompt, hypothetical_prompt2],
    "prompt_type":['Zero-shot prompting', 'Zero-shot prompting', 'Direct prompting', 'Direct prompting', 'Few-shot prompting', 'Few-shot prompting', 'COT prompting', 'COT prompting', 'Contextual prompting', 'Contextual prompting', 'Persona prompting', 'Persona prompting', 'Completion prompting', 'Completion prompting', 'Guided prompting', 'Guided prompting', 'Hypothetical prompting', 'Hypothetical prompting'],
    "response(with RAG)":[zs_response, zs_response2, direct_response, direct_response2, fs_response, fs_response2, cot_response, cot_response2, contextual_response, contextual_response2, persona_response, persona_response2, completion_response, completion_response2, guided_response, guided_response2, hypothetical_response, hypothetical_response2],
    "response(without RAG)":[zs_response_no_rag, zs_response_no_rag2, direct_response_no_rag, direct_response_no_rag2, fs_response_no_rag, fs_response_no_rag2, cot_response_no_rag, cot_response_no_rag2, contextual_response_no_rag, contextual_response_no_rag2, persona_response_no_rag, persona_response_no_rag2, completion_response_no_rag, completion_response_no_rag2, guided_response_no_rag, guided_response_no_rag2, hypothetical_response_no_rag, hypothetical_response_no_rag2]
}

results = pd.DataFrame(dict_results)

In [272]:
results

prompt             prompt_type  \
0   \nUse the following pieces of context to answe...     Zero-shot prompting   
1   \nWrite an email for XYZ Bank to promote their...     Zero-shot prompting   
2   \nGenerate a remittance promotion message with...        Direct prompting   
3   \nCompose an email for XYZ Bank that encourage...        Direct prompting   
4   \nGenerate remittance promotion messages for t...      Few-shot prompting   
5   \nGiven these examples of email marketing mess...      Few-shot prompting   
6   \nGuidelines:\nDesigning beaviourally-informed...           COT prompting   
7   \nGuidelines:\nDesigning beaviourally-informed...           COT prompting   
8   \n**Example:**\n- Context: "Bank XYZ is offeri...    Contextual prompting   
9   \nUse the following pieces of context to answe...    Contextual prompting   
10  \n**Example:**\n- Prompt: "You are a marketing...       Persona prompting   
11  \nAs a young professional looking to secure yo...       Persona prompting   
12  \n**Example:**\n- Prompt: " Make your money go...    Completion prompting   
13  \nXYZ Bank’s new investment plan is designed t...    Completion prompting   
14  \nUse the following pieces of context to answe...        Guided prompting   
15  \nUse the following pieces of context to answe...        Guided prompting   
16  \nImagine a customer looking to remit their mo...  Hypothetical prompting   
17  \nUse the following pieces of context to answe...  Hypothetical prompting   

                                   response(with RAG)  \
0   ## Subject: Invest Your Annual Bonus Wisely wi...   
1   ## Subject: Invest in Your Future with XYZ Ban...   
2   ## XYZ Remit: Your Gateway to China\n\nEnjoy c...   
3   ## Subject: Plan Your Future with XYZ Bank's I...   
4   ## Remittance Promotion Messages:\n\n**1. XYZ ...   
5   ## Subject: Diversify Your Portfolio and Maxim...   
6   ## Step 1: Analyze and understand the instruct...   
7   ## Step 1: Analyze and understand the instruct...   
8   ## Subject: Send money from Singapore to Malay...   
9   ## Subject: Plan Your Future with Our Wealth P...   
10  ## Subject: Enjoy $0 Transfer Fees to China wi...   
11  ## Subject: Invest in Your Future with XYZ Ban...   
12  ## Subject: Send money to Malaysia at preferen...   
13  ## Subject: Plan Your Future with Our Investme...   
14  ## Send Money Overseas with Ease and Save with...   
15  ## Subject: **Maximize Your Savings and Secure...   
16  ## Send Money Home with Confidence - Choose XY...   
17  ## Subject: Invest Your Bonus Wisely with XYZ ...   

                                response(without RAG)  
0   ## Subject: Invest Your Bonus Wisely with XYZ ...  
1   ## Subject: Invest with Confidence: Why XYZ Ba...  
2   ## Subject: XYZ Remit: Your Gateway to China\n...  
3   ## Subject: Unlock Your Financial Future with ...  
4   ## XYZ Remit: Your Gateway to China\n\n**Enjoy...  
5   ## Subject: Invest with Confidence: Manage Ris...  
6   ## Step 1: Analysis and Understanding\n\n**Goa...  
7   ## Step 1: Analyze and Understand the Instruct...  
8   ## Example 1:\n\n**Context:** "A new report by...  
9   ## Subject: Secure Your Future with Bank XYZ's...  
10  ## \n\n## Bank XYZ Remit: Send Money to China ...  
11  ## Subject: Secure Your Financial Future with ...  
12  - Prompt: "I have a 2015 Honda Civic EX-T with...  
13  ## XYZ Bank's New Investment Plan: \n\n**XYZ B...  
14  ## Send Money Overseas with Ease and Save Big ...  
15  ## Subject: Maximize Your Savings with XYZ Ban...  
16  ## XYZ Bank: Your Reliable Partner for Monthly...  
17  ## Subject: Invest Your Bonus Wisely with XYZ ...

In [273]:
results.to_csv('results.csv')